## Deep Classification Network

0. First, the grid established in the Dataset exploration notebook will be tested with the SVM. Then, a DNN is implemented.

1. Import all `modules` we are going to need

In [1]:
import torch
from torch import nn
import torchvision

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

2. Import Input Matrix X. Then, import y

In [3]:
import os
import pandas as pd

In [4]:
os.chdir("/mnt/c/Users/janos/OneDrive/Desktop/Master_thesis")

In [5]:
# the number of row in each data frame
# you can put any value here according to your situation
chunksize = 1000
index=0
# the list that contains all the dataframes
list_of_dataframes = []

for df in pd.read_csv('X_Matrix.csv', chunksize=chunksize):
    # process your data frame here
    # then add the current data frame into the list
    list_of_dataframes.append(df)
    print('Read the {num} dataframe'.format(num=index))
    index=index+1
# if you want all the dataframes together, here it is
#result = pd.concat(list_of_dataframes)

Read the 0 dataframe
Read the 1 dataframe
Read the 2 dataframe
Read the 3 dataframe
Read the 4 dataframe
Read the 5 dataframe
Read the 6 dataframe
Read the 7 dataframe
Read the 8 dataframe
Read the 9 dataframe
Read the 10 dataframe
Read the 11 dataframe
Read the 12 dataframe
Read the 13 dataframe
Read the 14 dataframe


3. Import y

In [6]:
os.chdir('/mnt/c/Users/janos/OneDrive/Desktop/Master_thesis/kosti/mat2csv/emotic_pre/')
train = pd.read_csv('train.csv')
train_mscoco = train[train['Folder'] == 'mscoco/images']

In [7]:
import re

In [8]:
y = []
i=0
for rating in train_mscoco['Continuous_Labels']:
    y.append(rating)

In [9]:
y_ = []
for idx, label in enumerate(y):
    s = (re.sub(r"[\[\]]",'',y[idx]))
    for i, x in enumerate(s):
        y_.append(x)
        if x == ',':
            y_.remove(x)
        if x == ' ':
            y_.remove(x)
        try:
            if x == '1' and s[i+1] == '0':
                y_.remove(x)
                y_.append('10')
        except IndexError:
            continue
        



In [10]:
for idx, el in enumerate(y_):
    if el == '10':
        if y_[idx+1] == '0':
            y_.remove(y_[idx+1])



In [11]:
len(y_)

42828

In [12]:
cont_labels = []
for val in y_:
    cont_labels.append(float(val))

In [13]:
img = 0
Arousal = []
Valence = []
Dominance = []
for val in cont_labels:
    if img == 0:
        Arousal.append(val)
        img = img+1
        continue
    if img == 1:
        Valence.append(val)
        img = img+1
        continue 
    if img == 2:
        Dominance.append(val)
        img = 0
        continue
    



In [14]:
import numpy as np

Arousal = np.array(Arousal)



In [15]:
Arousal.shape

(14276,)

In [16]:
y = Arousal

In [17]:
list_of_dataframes[14].shape[0]

276

In [18]:
from sklearn.feature_selection import SelectKBest

In [32]:
k_select = SelectKBest(k=40000)

In [19]:
y_df = pd.DataFrame(y)

In [20]:
y_series = y_df.squeeze()

In [21]:
y_series[14000:14276]

14000    7.0
14001    8.0
14002    7.0
14003    8.0
14004    8.0
        ... 
14271    5.0
14272    8.0
14273    2.0
14274    6.0
14275    7.0
Name: 0, Length: 276, dtype: float64

**NEXT STEP**:
`Rename`all columns more meaningful. Then apply k-best to all chunked dataframes.

In [22]:
for df in list_of_dataframes:
    for pos, col in enumerate(df):
        df.columns.values[pos] = 'Feature' + str(pos)
        

In [45]:
k_list = []
track = 0
i=0
j=1000
for chunk in list_of_dataframes:
    k_select.fit_transform(chunk, y_df[i:j])
    cols_idxs = k_select.get_support(indices=True)
    k_list.append(chunk.iloc[:,cols_idxs])
    print('Applied k-best to {track} chunks!'.format(track=track))
    track=track+1
    if track == 14:
        i=14000
        j=14000+list_of_dataframes[14].shape[0]
        
    else:
        i=i+1000
        j=j+1000
        

/home/jpauli/miniconda3/envs/yolo/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Applied k-best to 0 chunks!


/home/jpauli/miniconda3/envs/yolo/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Applied k-best to 1 chunks!


/home/jpauli/miniconda3/envs/yolo/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

Applied k-best to 12 chunks!


/home/jpauli/miniconda3/envs/yolo/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Applied k-best to 13 chunks!


/home/jpauli/miniconda3/envs/yolo/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Applied k-best to 14 chunks!


In [53]:
k_list[8]

,Feature0,Feature2,Feature6,Feature8,Feature9,Feature10,Feature11,Feature12,Feature17,Feature18,...,Feature50164,Feature50165,Feature50167,Feature50168,Feature50169,Feature50170,Feature50171,Feature50172,Feature50173,Feature50176
8000,8000,0.010968,0.005971,0.005024,0.006697,0.005194,0.006136,0.011277,0.001848,0.007945,...,0.054926,0.036575,0.012423,0.049717,0.031604,0.010351,0.008495,0.038107,0.062138,0.089240
8001,8001,0.076054,0.202234,0.088045,0.326959,0.200962,0.281703,0.112076,0.131658,0.295174,...,0.012347,0.021219,0.004784,0.003951,0.022180,0.006294,0.008456,0.005869,0.005350,0.009068
8002,8002,0.030585,0.121949,0.198508,0.085249,0.104548,0.116697,0.315565,0.061166,0.386085,...,0.022246,0.022030,0.008226,0.003169,0.003360,0.002436,0.016749,0.008145,0.040456,0.026661
8003,8003,0.030585,0.121949,0.198508,0.085249,0.104548,0.116697,0.315565,0.061166,0.386085,...,0.022246,0.022030,0.008226,0.003169,0.003360,0.002436,0.016749,0.008145,0.040456,0.026661
8004,8004,0.122884,0.029114,0.049784,0.083367,0.040112,0.019237,0.024253,0.036263,0.038604,...,0.012718,0.010536,0.001468,0.008702,0.009145,0.012497,0.018283,0.011756,0.004769,0.000354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,8995,0.076408,0.023384,0.015339,0.010341,0.017298,0.044009,0.037489,0.031711,0.020510,...,0.004593,0.001488,0.011672,0.006992,0.004873,0.001860,0.006952,0.005668,0.003266,0.022309
8996,8996,0.034827,0.060703,0.069598,0.023827,0.051291,0.027772,0.062071,0.114373,0.067908,...,0.007540,0.004574,0.004183,0.006079,0.006021,0.015051,0.012404,0.003840,0.009810,0.005647
8997,8997,0.047967,0.032935,0.012680,0.031420,0.027600,0.015465,0.038896,0.016858,0.019865,...,0.003303,0.008046,0.011015,0.014046,0.008323,0.010633,0.005889,0.020715,0.005079,0.001990
8998,8998,0.040696,0.072321,0.098541,0.215239,0.238613,0.061221,0.077495,0.131670,0.064194,...,0.099551,0.144208,0.177405,0.224425,0.102011,0.118900,0.223585,0.116678,0.267085,0.437534


Transform X to numpy array